In [2]:
import numpy as np
import cv2
import tensorflow as tf
import sklearn

2021-12-11 13:48:34.280845: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/site-packages/cv2/../../../../lib:
2021-12-11 13:48:34.280873: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
from os import listdir
from os.path import isfile, join

In [7]:
data_path = "../data/ai4mars-dataset-merged-0.1/msl/labels/train/"

onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

In [9]:
onlyfiles[0]

'NLB_512730161EDR_F0532980NCAM00265M1.png'